In [1]:
import os
import torch
from torch import nn
import numpy as np
import random
from dataset import ModelnetDataset
from pathlib import Path
from torch.utils.data import DataLoader


from util import setup_seed, train, test, train_HHN, test_HHN, train_one4one, count_parameters, save_checkpoint
from model import One4All, HyperNetwork
from render import PointcloudRender

from torch.utils.tensorboard import SummaryWriter

In [3]:
setup_seed(666)
device_for_dataset = "cuda:0" if torch.cuda.is_available() else "cpu"
writer = SummaryWriter('tf-logs/one4all-12-03')


In [4]:
path = Path("./data/ModelNet10")
train_ds = ModelnetDataset(path, folder="train", device=device_for_dataset) # Just put dataset in mem or GPU
test_ds = ModelnetDataset(path, folder="test", device=device_for_dataset) # Just put dataset in mem or GPU
classes = {i: cat for cat, i in train_ds.classes.items()}

print('Train dataset size: ', len(train_ds))
print('Test dataset size: ', len(test_ds))
print('Number of classes: ', len(train_ds.classes))
print('Class: ', classes[train_ds[11]['category']])

Preocessing train data in_mem = True and infor: Default infor


Prepare train dataset categories:   0%|          | 0/10 [00:00<?, ?it/s]

Preocessing test data in_mem = True and infor: Default infor


Prepare test dataset categories:   0%|          | 0/10 [00:00<?, ?it/s]

Train dataset size:  3991
Test dataset size:  908
Number of classes:  10
Class:  bathtub


In [5]:
from collections import Counter
print(dict(Counter(sample['category'] for sample in train_ds.files)))

print(dict(Counter(sample['category'] for sample in test_ds.files)))



{'bathtub': 106, 'bed': 515, 'chair': 889, 'desk': 200, 'dresser': 200, 'monitor': 465, 'night_stand': 200, 'sofa': 680, 'table': 392, 'toilet': 344}
{'bathtub': 50, 'bed': 100, 'chair': 100, 'desk': 86, 'dresser': 86, 'monitor': 100, 'night_stand': 86, 'sofa': 100, 'table': 100, 'toilet': 100}


In [6]:
batch_size = 256 #  
epochs = 6000 #
best_acc1 = 0
start_epoch = 0


In [7]:
train_loader = DataLoader(dataset=train_ds, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_ds, batch_size=batch_size, shuffle=True)


In [8]:
best_filename = 'checkpoint/one4all/12-03/best-one4allcheckpoint.pth.tar'

gpu_computation = "cuda:0" if torch.cuda.is_available() else "cpu"

model = One4All() 
# model = HyperNetwork(6)#
model = model.to(gpu_computation)
print(f'Num of model parameter: {count_parameters(model)}')
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.002, weight_decay = 1e-6)
# optimizer = torch.optim.AdamW(model.parameters(), lr=0.004)
# optimizer = torch.optim.SGD(model.parameters(), 0.001,
#                                 momentum=0.9,
#                                 weight_decay=1e-4
#                            )

# milestones = [30, 60, 150, 200]
# lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer=optimizer, milestones=milestones, gamma=0.5)
lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs+6000, eta_min=5e-6)
pr = PointcloudRender(              
                 img_size=128,
                 radius=0.0045,
                 points_per_pixel=40)


print(model)


Num of model parameter: 138562
=> loading checkpoint 'checkpoint/one4all/12-02/12-02-NO5950-one4allcheckpoint.pth.tar'
=> loaded checkpoint 'checkpoint/one4all/12-02/12-02-NO5950-one4allcheckpoint.pth.tar' (epoch 5951)
One4All(
  (con_block): Sequential(
    (0): Conv2d(1, 4, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(4, 16, kernel_size=(5, 5), stride=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Sequential(
    (0): Linear(in_features=400, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=10, bias=True)
  )
)


In [ ]:
torch.cuda.empty_cache()


angles = torch.randint(-180, 180, (3,))/180*torch.pi


for t in range(start_epoch, epochs):
# for t in range(start_epoch, epochs):
    print(60*"="+f"\n Epoch: {t + 1} \n"+60*"=")
    # angles = torch.randint(-180, 180, (3,))/180*torch.pi
    
    train_acc, train_loss= train(train_loader, model, loss_fn, optimizer, t, pr, device=gpu_computation)
    test_acc, test_loss = test(test_loader, model, loss_fn, pr, device=gpu_computation, angles=angles)
    writer.add_scalar("Loss/train", train_loss, t)
    writer.add_scalar("Accuracy/train", train_acc, t)
    
    writer.add_scalar("Loss/test", test_loss, t)
    writer.add_scalar("Accuracy/test", test_acc, t)
    
    is_best = test_acc > best_acc1
    best_acc1 = max(test_acc, best_acc1)

    if t%50 == 0:
        save_checkpoint({
        'epoch': t + 1,
        'state_dict': model.state_dict(),
        'best_acc1': best_acc1,
        'optimizer': optimizer.state_dict(),
        }, is_best,
        filename= f'checkpoint/one4all/12-03/12-03-NO{t}-one4allcheckpoint.pth.tar', #saved_filename, #Todo add GPU and epoch information to the filename
        best_filename = best_filename
        )
    writer.flush()
    lr_scheduler.step()

print("Done!")
writer.close()